In [1]:
# extract Z, y, X from covariates, expression and genotype data and generate y_res
[extract]
parameter: ensembl_gene_id = str
bash: expand = True
    # y
    zless /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_expression_matrices/*.gz | grep {ensembl_gene_id} > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}.txt
    # X
    awk '$4 ~ /{ensembl_gene_id}/ {{print $0}}' /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/gene_TSS.csv | head -n 1 | awk '{{$3=$2+1000000}} {{$2=$2-1000000}} {{print "chr"$1":"$2"-"$3}}' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_TSSregion.txt
    tabix /project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/genotypes/WGS/variant_calls/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze.SHAPEIT2_phased.vcf.gz "$(< /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_TSSregion.txt)" | sed -e 's/0|0/0/g' -e 's/0|1/1/g' -e 's/1|0/1/g' -e 's/1|1/2/g' > /scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_genotype.txt
    
R: expand = True
    # Z
    filenames <- list.files(path = "/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_covariates", pattern = "*.txt", full.names = TRUE)
    Z <- lapply(filenames, function(x) t(as.matrix(read.table(file = x, header = TRUE, sep = '\t', quote = "", row.names = 1))))
    
    # y
    y_raw <- scan(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}.txt", what = "", sep = "\n")
    y <- strsplit(x = y_raw, split = "[[:space:]]+")
    y <- lapply(y, function(x) x[-1:-4])
    y <- lapply(y, as.numeric)
    
    # X
    X <- read.csv(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/{ensembl_gene_id}_genotype.txt", sep = '\t', header = FALSE, row.names = 3, stringsAsFactors = FALSE)
    X <- X[,-(1:8)]
    samplenames_X <- scan(file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/samplenames_X.txt", what = character(), quote = "")
    samplenames_X <- samplenames_X[-(1:9)]
    colnames(X) <- samplenames_X
    X <- as.matrix(x = X)
    X <- t(X)
    
    # y_res
    y_res <- lapply(1:49, function(i) .lm.fit(x = Z[[i]], y = y[[i]])$residuals)

    # save
    saveRDS(object = list(X = X, y = y, Z = Z, y_res = y_res), file = "/scratch/midway2/chj1ar/GTEx_Analysis_v8_eQTL_expression_genewise/Multi_Tissues.{ensembl_gene_id}.RDS")

